In [ ]:
import os

iv = os.urandom(16)
print(iv)

b'@\x01>n\xd2\x12\x04\xc2\xf4\x0bS\xe7\x89\x1b\xfd%'


**CBC Encryption**

---

params:
- bit_input = 16 bit binary message
- key = 4 bit binary key
- iv = 4 bit binary *IV* constant

output:
- encrypted_message = CBC-encrypted 16 bit binary message
   

In [ ]:
def xor(a, b):
    return ''.join(str(int(x) ^ int(y)) for x, y in zip(a, b))

def cbc_encrypt_4bit(key, iv, bit_input):

    assert len(key) == 4, "Key must be 4 bits."
    assert len(iv) == 4, "IV must be 4 bits."
    assert len(bit_input) == 16, "Input must be 16 bits."

    # Split the input into four 4-bit parts
    parts = [bit_input[i:i+4] for i in range(0, len(bit_input), 4)]
    # for part in parts:
    #   print(part)

    encrypted_parts = []
    current_iv = iv

    for part in parts:
        # XOR with IV
        xor_with_iv = xor(part, current_iv)
        # print(len(xor_with_iv))
        # XOR with key
        encrypted_part = xor(xor_with_iv, key)
        # append encrypted part to result
        encrypted_parts.append(encrypted_part)
        # update IV for the next block
        current_iv = encrypted_part

    # concat all encrypted parts to form the final 16-bit encrypted message
    encrypted_message = ''.join(encrypted_parts)

    return encrypted_message


key = '1010'
iv = '1100'
bit_input = '1100110010101010'

# function testing
encrypted_message = cbc_encrypt_4bit(key, iv, bit_input)

print(f'Original bit input: {bit_input}')
print(f'key: {key}')
print(f'iv: {iv}')
print(f'Encrypted message: {encrypted_message}')

Original bit input: 1100110010101010
key: 1010
iv: 1100
Encrypted message: 1010110011001100


**Feistel Encryption**

---

params:
- bit_input = 16 bit binary message
- key = 4 bit binary key

output:
- output_bitstring = Feistel-encrypted 16 bit binary message

In [ ]:
def feistel_round(L, R, key):

    # XOR operation for the round function, with bitstrings
    new_L = R
    new_R = format(int(L, 2) ^ (int(R, 2) ^ int(key, 2)), '08b')
    return new_L, new_R

def feistel_block(bit_input, key, rounds=16):

    if len(bit_input) != 16:
        raise ValueError("Input bitstring must be exactly 16 bits")

    # split input into two 8-bit halves
    L = bit_input[:8]
    R = bit_input[8:]

    if len(key) != 8:
        raise ValueError("Key bitstring must be exactly 8 bits")

    # perform Feistel rounds
    for i in range(rounds):
        L, R = feistel_round(L, R, key)

    # concat the halves back into a 16-bit output
    output_bitstring = L + R

    return output_bitstring


bit_input = '1100110010101010'
feistel_key = '01100110'

encrypted_output = feistel_block(bit_input, feistel_key, rounds=16)

print(f'Input: {bit_input}')
print(f'Feistel Key: {feistel_key}')
print(f'Encrypted Output: {encrypted_output}')

Input: 1100110010101010
Feistel Key: 01100110
Encrypted Output: 1010101000000000


**Bit-by-Bit XOR Function**

---
params:
- input_16bit = 16 bit binary message

output:
- xor_result = 8 bit XOR-ed message

In [ ]:
def xor_8bit(input_16bit):

    assert len(input_16bit) == 16, "Input must be 16 bits."

    # split the input into two 8-bit parts
    part1 = input_16bit[:8]
    part2 = input_16bit[8:]

    # perform XOR on the two parts
    xor_result = ''.join(str(int(x) ^ int(y)) for x, y in zip(part1, part2))

    return xor_result


input_16bit = '1100110010101010'

xor_result = xor_8bit(input_16bit)

print(f'Original 16-bit input: {input_16bit}')
print(f'XOR result of the two 8-bit parts: {xor_result}')


Original 16-bit input: 1100110010101010
XOR result of the two 8-bit parts: 01100110


**Proposed Hash Algorithm**

---


1. CBC 16 bits
2. Feistel 16 bits, 16 rounds
3. Bit-by-Bit XOR, returns an 8 bit encrypted message

In [ ]:
def hash_encrypt(key, iv, message, feistel_key):

    cbc_result = cbc_encrypt_4bit(key, iv, message)
    feistel_result = feistel_block(cbc_result, feistel_key, rounds=16)
    xor_res = xor_8bit(feistel_result)

    return xor_res

**Proposed and Modified HMAC Function**

---

In [ ]:
def hmac_custom(message, hash_key, key, feistel_key, iv):
    ipad = '00110110'
    opad = '01011100'

    # XOR operation for inside and outside hash
    inside_hash_bin = xor(hash_key, ipad)
    outside_hash_bin = xor(hash_key, opad)

    # concat message and inside hash, and pad if necessary
    concat_message_inside_hash = message + inside_hash_bin

    if len(concat_message_inside_hash) < 16:
        concat_message_inside_hash = concat_message_inside_hash.ljust(16, '0')

    # encrypt with proposed hash algorithm
    xor_res = hash_encrypt(key, iv, concat_message_inside_hash, feistel_key)

    # concat with outside hash, and pad if necessary
    concat_feistel_outside_hash = xor_res + outside_hash_bin
    if len(concat_feistel_outside_hash) < 16:
        concat_feistel_outside_hash = concat_feistel_outside_hash.ljust(16, '0')

    final_xor_res = hash_encrypt(key, iv, concat_feistel_outside_hash, feistel_key)

    return final_xor_res

message = '11001100'
feistel_key = '01100110'
hash_key = '11001100'
key = '1100'
iv = '1100'

hmac_result = hmac_custom(message, hash_key, key, feistel_key, iv)

print(f'HMAC Result (8 bits): {hmac_result}')

HMAC Result (8 bits): 11001010


**Finding Collisions: First block of HMAC**

---

In [ ]:
import itertools

# Find Collisions Function
def find_collision(hash_key, key, iv, feistel_key, func):
    messages = [''.join(seq) for seq in itertools.product('01', repeat=8)]
    results = {}

    for message in messages:
        result = func(message, hash_key, key, feistel_key, iv)
        if result in results:
            return results[result], message
        results[result] = message

    return None, None

# The first hash block of the HMAC function
def hash_block_1(message, hash_key, key, feistel_key, iv):
    ipad = '00110110'

    inside_hash_bin = xor(hash_key, ipad)
    concat_message_inside_hash = message + inside_hash_bin

    if len(concat_message_inside_hash) < 16:
        concat_message_inside_hash = concat_message_inside_hash.ljust(16, '0')

    xor_res = hash_encrypt(key, iv, concat_message_inside_hash, feistel_key)

    return xor_res

msg1, msg2 = find_collision(hash_key, key, iv, feistel_key, hash_block_1)

if msg1 and msg2:
    print(f'Found collision:\nMessage 1: {msg1}\nMessage 2: {msg2}\nXOR result: {hash_block_1(msg1, hash_key, key, feistel_key, iv)}')
else:
    print('No collision found.')

No collision found.


**Finding Collisions: Hash Function**

---

In [ ]:
def find_collision_trunc(key, iv, feistel_key):
    messages = [''.join(seq) for seq in itertools.product('01', repeat=16)]
    results = {}

    for message in messages:
        result = hash_encrypt(key, iv, message, feistel_key)
        if result in results:
            return results[result], message
        results[result] = message

    return None, None

msg1, msg2 = find_collision_trunc(key, iv, feistel_key)

if msg1 and msg2:
    print(f'Found collision:\nMessage 1: {msg1}\nMessage 2: {msg2}\nXOR result: {hash_encrypt(key, iv, msg1, feistel_key)}')
else:
    print('No collision found.')

Found collision:
Message 1: 0000000000000000
Message 2: 0000000000000001
XOR result: 01101010


**Finding Collisions: HMAC**

---

In [ ]:
msg1, msg2 = find_collision(hash_key, key, iv, feistel_key, hmac_custom)

if msg1 and msg2:
    print(f'Found collision:\nMessage 1: {msg1}\nMessage 2: {msg2}\nXOR result: {hmac_custom(msg1, hash_key, key, feistel_key, iv)}')
else:
    print('No collision found.')

No collision found.


In [ ]:
message = '11001100'
feistel_key = '01100110'
hash_key = '11001100'
key = '1100'
iv = '1100'


**User Input HMAC Encryption**


---



In [ ]:
def main():
    message = input("Enter the message (8 bits): ")
    hash_key = input("Enter hash key (8 bits): ")
    feistel_key = input("Enter feistel key (8 bits): ")
    key = input("Enter the key (4 bits): ")
    iv = input("Enter the IV constant (4 bits): ")

    hmac_result = hmac_custom(message, hash_key, key, feistel_key, iv)
    print()
    print("HMAC RESULT:", hmac_result)

if __name__ == "__main__":
    main()

Enter the message (8 bits): 10101010
Enter hash key (8 bits): 00011010
Enter feistel key (8 bits): 11010011
Enter the key (4 bits): 1111
Enter the IV constant (4 bits): 0011

HMAC RESULT: 10101011
